In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)
stemmer = PorterStemmer()

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [3]:
tokenize('how are you?')

['how', 'are', 'you', '?']

In [4]:
def stem(word):
    return stemmer.stem(word.lower())

In [5]:
stem('ORGANIC')

'organ'

In [6]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(tokenize('how are you ha ha organic organ'),['organ','story','numpy','organix','ha','are'])

array([1., 0., 0., 0., 1., 1.], dtype=float32)

In [8]:
data = pd.read_csv('./data.csv')

In [9]:
X = data['MESSAGE']
y = data['CATEGORY']

In [10]:
all_words = []
all_data = []
tags = []

In [11]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_bacth = tokenize(X_batch)
    new_X = []
    for X_batch_in_X_batch in X_bacth:
        new_X.append(stem(X_batch_in_X_batch))
    all_words.extend(new_X)
    all_data.append((new_X,y_batch))
    tags.append(y_batch)

5796it [00:35, 162.18it/s]


In [12]:
len(all_words)

3054626

In [13]:
len(all_data)

5796

In [14]:
len(tags)

5796

In [15]:
all_words = sorted(set(all_words))

In [16]:
len(all_words)

93148

In [17]:
tags = sorted(set(tags))

In [18]:
len(tags)

2

In [19]:
X = []
y = []

In [ ]:
for sentence,tag in tqdm(all_data):
    sentence = bag_of_words(sentence,all_words)
    X.append(sentence)
    y.append(tags.index(tag))

 16%|██████▎                                 | 922/5796 [07:44<43:15,  1.88it/s]

In [ ]:
y[0]

In [ ]:
X[0]

In [ ]:
from sklearn.model_selection import *

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [ ]:
device = 'cuda'

In [ ]:
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device)
y_test = torch.from_numpy(np.array(y_test)).to(device)

In [ ]:
torch.save(X_train,'X_train.pth')
torch.save(X_test,'X_test.pth')
torch.save(y_train,'y_train.pth')
torch.save(y_test,'y_test.pth')
torch.save(all_data,'all_data.pth')
torch.save(all_words,'all_words.pth')
torch.save(tags,'tags.pth')
X_train = torch.load('X_train.pth').to(device).float()
y_train = torch.load('y_train.pth').to(device).float()
X_test = torch.load('X_test.pth').to(device).float()
y_test = torch.load('y_test.pth').to(device).float()
all_data = list(torch.load('all_data.pth'))
all_words = list(torch.load('all_words.pth'))
tags = list(torch.load('tags.pth'))

In [ ]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.activation = ReLU()
        self.iters = 5
        self.linear1 = Linear(len(all_words),512)
        self.linear2 = Linear(512,512)
        self.linear3 = Linear(512,1)
        self.linear3activation = Sigmoid()
    
    def forward(self,X):
        preds = self.activation(self.linear1(X))
        for iteration in range(self.iters):
            preds = self.activation(self.linear2(preds))
        preds = self.linear3activation(self.linear3(preds))
        return preds

In [ ]:
model = Model().to(device)

In [ ]:
criterion = MSELoss()

In [ ]:
optimizer = Adam(model.parameters(),lr=0.001)

In [ ]:
epochs = 100

In [ ]:
batch_size = 32

In [ ]:
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].to(device).float()
        y_batch = y_train[idx:idx+batch_size].to(device).float()
        preds = model(X_batch)
        preds = preds.to(device)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
import wandb

In [ ]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [ ]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.round(pred))
        y_batch = int(torch.round(y_batch))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [ ]:
wandb.init(project='Spam-Clf-V1',name='baseline')
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].to(device).float()
        y_batch = y_train[idx:idx+batch_size].to(device).float()
        preds = model(X_batch)
        preds = preds.to(device)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+loss.item()+get_loss(model,X_batch,y_batch,criterion))/3})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train,criterion)+get_accuracy(model,X_batch,y_batch,criterion))/2})
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test,criterion)})
wandb.finish()